In [ ]:
from pyspark.sql import functions as F
from delta.tables import DeltaTable

TABELA_ORIGEM_TRACK_ARTISTS = "spotify_analytics.silver.tb_track_artists"
TABELA_ORIGEM_TRACKS = "spotify_analytics.silver.tb_tracks"
TABELA_DESTINO = "spotify_analytics.gold.dm_artist"

### Configuração

Define origens (Silver tb_track_artists + tb_tracks) e destino (Gold dm_artist).

### Transformação Silver → Gold (Dimensão Artist)

Lê dados do Silver e cria dimensão de artistas com:
1. **Agregação**: Conta quantidade de tracks por artista
2. **Popularidade Média**: Calcula popularidade média das tracks do artista
3. **Coluna Derivada de Negócio**: 
   - `ds_popularidade` baseada na popularidade média:
     - < 20: "Desconhecido"
     - 20-50: "Emergente"
     - 50-70: "Popular"
     - 70-90: "Muito Popular"
     - > 90: "Superstar"

**Decisão Arquitetural**: Calcular métricas agregadas na dimensão:
- **Vantagem**: Queries de BI mais simples (não precisa JOIN com fato para ver total de tracks)
- **Desvantagem**: Duplicação de informação
- **Trade-off**: Aceitável pois dimensões são pequenas e read-heavy

Faz MERGE em dm_artist (Gold).

In [ ]:
# Lê tabelas Silver
df_track_artists = spark.read.table(TABELA_ORIGEM_TRACK_ARTISTS)
df_tracks = spark.read.table(TABELA_ORIGEM_TRACKS)

# JOIN para obter popularidade das tracks
df_artist_popularity = (
    df_track_artists
    .join(df_tracks, on="track_id", how="inner")
    .groupBy("artist_id", "artist_name")
    .agg(
        F.count("track_id").alias("qt_tracks"),
        F.avg("popularity").alias("avg_popularity")
    )
)

# Cria dimensão com coluna derivada
df_gold = df_artist_popularity.select(
    F.col("artist_id").alias("id_artist_spotify"),
    F.col("artist_name").alias("nm_artist"),
    F.col("qt_tracks"),
    
    # Coluna derivada de negócio
    F.when(F.col("avg_popularity") < 20, "Desconhecido")
     .when(F.col("avg_popularity") < 50, "Emergente")
     .when(F.col("avg_popularity") < 70, "Popular")
     .when(F.col("avg_popularity") < 90, "Muito Popular")
     .otherwise("Superstar")
     .alias("ds_popularidade")
)

print(f"Total de artistas: {df_gold.count()}")
df_gold.orderBy(F.desc("qt_tracks")).show(10, truncate=False)

### MERGE em dm_artist

**Importante**: Não fazemos match por cd_artist (surrogate key autoincrement), mas sim por id_artist_spotify (natural key).

**Decisão Arquitetural**: 
- MATCHED: Atualiza dados (qt_tracks pode mudar)
- NOT MATCHED: Insere novo artista (cd_artist será gerado automaticamente)

In [ ]:
DeltaTable.forName(spark, TABELA_DESTINO).alias("t").merge(
    df_gold.alias("s"),
    "t.id_artist_spotify = s.id_artist_spotify"
).whenMatchedUpdate(
    set = {
        "nm_artist": "s.nm_artist",
        "qt_tracks": "s.qt_tracks",
        "ds_popularidade": "s.ds_popularidade"
    }
).whenNotMatchedInsert(
    values = {
        "id_artist_spotify": "s.id_artist_spotify",
        "nm_artist": "s.nm_artist",
        "qt_tracks": "s.qt_tracks",
        "ds_popularidade": "s.ds_popularidade"
    }
).execute()

print(f"✅ Dimensão Artist carregada!")

### Verificação Final

In [ ]:
spark.sql(f"""
    SELECT ds_popularidade, COUNT(*) as qtd_artistas
    FROM {TABELA_DESTINO}
    GROUP BY ds_popularidade
    ORDER BY qtd_artistas DESC
""").show()